# Linkage Disequilibrium

In [1]:
import hail as hl
hl.init()

Running on Apache Spark version 2.4.6
SparkUI available at http://hms-beagle-5466c684ff-2l8nm:4047
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.58-3f304aae6ce2
LOGGING: writing to /home/olavur/experiments/2020-11-13_fargen1_exome_analysis/fargen-1-exome/notebooks/hail-20201203-1610-0.2.58-3f304aae6ce2.log


In [184]:
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, ColumnDataSource
from bokeh.layouts import gridplot
from bokeh.models import FuncTickFormatter, ColorBar, LinearColorMapper, BasicTicker
output_notebook()

Loading BokehJS ...

In [4]:
mt = hl.read_matrix_table('/home/olavur/experiments/2020-11-13_fargen1_exome_analysis/data/mt/filtered.mt')

In [8]:
mt = mt.filter_rows(hl.len(mt.alleles) == 2)

In [9]:
mt.count()

(176834, 48)

In [197]:
mt_small = mt.filter_rows((mt.locus.contig == 'chr22') & (mt.locus.position < 15000000))

In [312]:
ld = hl.ld_matrix(mt_small.GT.n_alt_alleles(), mt_small.locus, radius=2e6)
ld = ld.to_numpy()

nn, _ = ld.shape
nn

2020-12-04 12:07:39 Hail: INFO: Wrote all 1 blocks of 91 x 48 matrix with block size 4096.


91

In [309]:
# Get position and RSID of each site.
pos = mt_small.locus.position.collect()
rsid = mt_small.rsid.collect()

# Prepare them as nn x nn matrices.
pos_mat = np.array([pos] * nn)
rsid_mat = np.array([rsid] * nn)

In [338]:
source = ColumnDataSource(data=dict(
    image=[ld],
    x=[0],  # Axis starts at 0.
    y=[0],
    dw=[nn-1],  # Axis has same dimension as LD matrix.
    dh=[nn-1],
    pos=[pos_mat],
    rsid=[rsid_mat]
))

TOOLTIPS = [
    ("(x,y)", "($x, $y)"),
    ("rho", "@image"),
    ("pos", "@pos"),
    ("rsid", "@rsid"),
]

p = figure(tooltips=TOOLTIPS)

# Add colorbar to plot.
color_mapper = LinearColorMapper(palette="Viridis256", low=np.nanmin(ld), high=np.nanmax(ld))
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=5, border_line_color=None, location=(0,0))
p.add_layout(color_bar, 'right')

# Provide the image, the annotations, and other parameters.
p.image(source=source, image='image', x='x', y='y', dw='dw', dh='dh', color_mapper=color_mapper)

# Remove the padding in the sides of the plot.
p.x_range.range_padding = p.y_range.range_padding = 0

# Use the position of the site as the axis ticks.
# The current ticks should be the index in the image (matrix), as we've set the axis to start at 0
# and end at nn-1. Therefore, we just map the current ticks to the position array.
pos_tick_format = FuncTickFormatter(args={'pos': pos}, code="""
    return pos[tick]
""")
p.xaxis.formatter = pos_tick_format
p.yaxis.formatter = pos_tick_format

p.plot_width = 700
p.plot_height = 500

show(p)